In [ ]:
from pydub import AudioSegment
import glob
import os

In [ ]:
def ExplodeCombination(files):
    output_dir = os.path.join(os.getcwd(), "samples")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
        
    wave_names = []
    sounds = []
    max_length = 0
    for f in files:
        sound = AudioSegment.from_file(f, format="wav")
        sounds.append(sound)
        name = os.path.basename(f)
        wave_names.append(name[:name.index('.')])
        print(len(sound))
        max_length = max(max_length, len(sound))

    print(max_length)
    #for f in wave_names:
    #    print(f)
        
    #for s in sounds:
    #    print(s)
    samples = []    
    total_combi = 2 ** len(wave_names)
    for i in range(total_combi):
        name = None
        mix = AudioSegment.silent(duration=max_length)
        for j in range(len(wave_names)):
            if i & (1 << j) != 0:
                mix = mix.overlay(sounds[j])
                if name == None:
                    name = wave_names[j]
                else:
                    name += ("-" + wave_names[j])
        samples.append((name, mix))
        if name != None:
            print(name, mix, len(mix))
            mix.export(os.path.join(output_dir, name) + ".wav", format="wav")
    return

def GenerateTraingData(source_dir):
    cwd = os.getcwd()
    files = glob.glob(os.path.join(cwd, source_dir, "*.wav"))
    #print(files)
    ExplodeCombination(files)
    return


In [ ]:
GenerateTraingData("audio_data")